In [ ]:
"""
for each folder of name sub-xxx:
	for each file in sub-xxx/eeg:
		if filename contains seqLearning**:
			#calculate cognitive score from the .tsv file 
			if filename ends with _events.tsv
				for each row in filename
					if “value” column is “dot_no7_off” 
						#calculate the cognitive score by comparing the user_answer and correct_answer
						#save the cognitive score and “onset” value (timestamp) 

"""

#load data from a single file for now


path = '/scratch/jsheikh2/ted_temp/ds005505/sub-NDARGJ395FKP/eeg/sub-NDARGJ395FKP_task-seqLearning8target_events.tsv'
#https://mne.tools/stable/generated/mne.io.read_raw_eeglab.html
import csv

import pandas as pd

df = pd.read_csv(path, sep='\t')

### For some reason, starting from the SECOND occurence of dot 7 OFF is where we get scores
display(df[0:24])

,onset,duration,sample,value,event_code,user_answer,correct_answer,target_count
0,0.000,NaN,0.0,break cnt,break cnt,NaN,NaN,8
1,40.846,NaN,20423.0,seqLearning_start,91,NaN,NaN,8
2,47.132,NaN,23566.0,learningBlock_1,31,NaN,NaN,8
3,47.150,NaN,23575.0,dot_no1_ON,11,NaN,NaN,8
4,47.740,NaN,23870.0,dot_no1_OFF,21,NaN,NaN,8
5,49.058,NaN,24529.0,dot_no6_ON,16,NaN,NaN,8
6,49.638,NaN,24819.0,dot_no6_OFF,26,NaN,NaN,8
7,50.958,NaN,25479.0,dot_no4_ON,14,NaN,NaN,8
8,51.538,NaN,25769.0,dot_no4_OFF,24,NaN,NaN,8
9,52.858,NaN,26429.0,dot_no2_ON,12,NaN,NaN,8


In [7]:
print(df['value'].unique())

['break cnt' 'seqLearning_start' 'learningBlock_1' 'dot_no1_ON'
 'dot_no1_OFF' 'dot_no6_ON' 'dot_no6_OFF' 'dot_no4_ON' 'dot_no4_OFF'
 'dot_no2_ON' 'dot_no2_OFF' 'dot_no7_ON' 'dot_no7_OFF' 'dot_no3_ON'
 'dot_no3_OFF' 'dot_no8_ON' 'dot_no8_OFF' 'dot_no5_ON' 'dot_no5_OFF'
 'learningBlock_2' 'learningBlock_3' 'learningBlock_4' 'learningBlock_5'
 'seqLearning_stop' 'boundary']


In [11]:
df.loc[df['value']== 'dot_no7_OFF']

,onset,duration,sample,value,event_code,user_answer,correct_answer,target_count
12,55.348,NaN,27674.0,dot_no7_OFF,27,NaN,NaN,8
22,64.858,NaN,32429.0,dot_no7_OFF,27,1-6-4-7-2-5-3-8-6-1,1-6-4-2-7-3-8-5-2-7,8
33,114.378,NaN,57189.0,dot_no7_OFF,27,NaN,NaN,8
43,123.878,NaN,61939.0,dot_no7_OFF,27,1-6-4-2-7-3-8-5-1-6,1-6-4-2-7-3-8-5-2-7,8
54,164.166,NaN,82083.0,dot_no7_OFF,27,NaN,NaN,8
64,173.666,NaN,86833.0,dot_no7_OFF,27,1-6-4-2-7-3-8-6-5-3,1-6-4-2-7-3-8-5-2-7,8
75,215.586,NaN,107793.0,dot_no7_OFF,27,NaN,NaN,8
85,225.106,NaN,112553.0,dot_no7_OFF,27,1-6-4-2-7-3-8-5-2-7,1-6-4-2-7-3-8-5-2-7,8
96,264.044,NaN,132022.0,dot_no7_OFF,27,NaN,NaN,8
106,273.564,NaN,136782.0,dot_no7_OFF,27,1-6-4-2-7-3-8-5-2-7,1-6-4-2-7-3-8-5-2-7,8


In [16]:
def calculate_score(df):
    """
    Calculate score for every Score Event (i.e. when user_answer and correct_answer are not NaN)
    Score is based on positional matching:
    - Compare user_answer and correct_answer position by position (same index)
    - Find the first contiguous subsequence of length > 1 where elements match at the same positions
    - Score is the length of this subsequence (0 if no match > 1 found)
    
    Returns a df containing only rows with valid user_answer and correct_answer, plus a 'score' column
    """
    import pandas as pd
    
    # Filter to only rows where both user_answer and correct_answer are not NaN
    mask = df['user_answer'].notna() & df['correct_answer'].notna()
    result_df = df[mask].copy()
    
    # Initialize score column
    result_df['score'] = 0
    
    for idx in result_df.index:
        user_ans = str(result_df.loc[idx, 'user_answer']).split('-')
        correct_ans = str(result_df.loc[idx, 'correct_answer']).split('-')
        
        # Ensure both lists have the same length (use minimum)
        min_len = min(len(user_ans), len(correct_ans))
        
        score = 0
        i = 0
        
        while i < min_len:
            # Count consecutive matches starting at position i
            match_length = 0
            j = i
            while j < min_len and user_ans[j] == correct_ans[j]:
                match_length += 1
                j += 1
            
            # If we found a match of length > 1, that's our score
            if match_length > 1:
                score = match_length
                break
            
            # Move to next position
            i += 1
        
        result_df.loc[idx, 'score'] = score
    
    return result_df

In [17]:
calculate_score(df)

,onset,duration,sample,value,event_code,user_answer,correct_answer,target_count,score
22,64.858,NaN,32429.0,dot_no7_OFF,27,1-6-4-7-2-5-3-8-6-1,1-6-4-2-7-3-8-5-2-7,8,3
43,123.878,NaN,61939.0,dot_no7_OFF,27,1-6-4-2-7-3-8-5-1-6,1-6-4-2-7-3-8-5-2-7,8,8
64,173.666,NaN,86833.0,dot_no7_OFF,27,1-6-4-2-7-3-8-6-5-3,1-6-4-2-7-3-8-5-2-7,8,7
85,225.106,NaN,112553.0,dot_no7_OFF,27,1-6-4-2-7-3-8-5-2-7,1-6-4-2-7-3-8-5-2-7,8,10
106,273.564,NaN,136782.0,dot_no7_OFF,27,1-6-4-2-7-3-8-5-2-7,1-6-4-2-7-3-8-5-2-7,8,10
